In [1]:
import inro.modeller as m
import os
import glob
import csv

mm = m.Modeller()
eb = m.Modeller().emmebank

#tools
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")           
change_matrix = mm.tool("inro.emme.data.matrix.change_matrix_properties")
create_matrix =  mm.tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = mm.tool("inro.emme.matrix_calculation.matrix_calculator")
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')

#HDR Scripts
common = mm.module('org.emme.common')
moe = mm.module('org.emme.moe')

#scenario folder names
sc_2011 = "2011 GGHM validation run"
sc_2041_base = "2041\\MTO_2041_GGH_MTP_328_V105_CS222_20190522"
sc_2041_donothing = "2041\\DoNothing scenario"
sc_2016 = "2016"
#sc_2041_C_plus = "2041\\MTO_2041_GGH_MTP_302_V103_CS221_20180601"
#sc_2041_C_plus_w_GTAWest = "2041\\MTP_304"
#sc_2041_base_new = "2041\\GGHM Base Case\\USE - Sent Through FTP September 20 2018"
#sc_2041_C_plus_w_GTAWest_new = "2041\\Committed_Plus"
#sc_2016_new = "2016\\MTO_2016_GGHM_RC61_cs211_GGH_MTP_20171031"

## zone grouping definitions ## 

zones_pd1 = "gf1"
zones_airports = "ga1"
zones_rural = "gb1"
zones_urban = "gb2;gb3;gb4"
zones_ugc = "gb4"
zones_inner = "gc1"
zones_outer = "gc2"
zones_university = "gd1"
zones_employment = "ge1"
zones_hospitals = "gh1"
shed_central = "gs10;gs11"
shed_central_urban = "gs10" # without pd1
shed_central_rural = "gs11"
shed_east = "gs20;gs21"
shed_east_urban = "gs20"
shed_east_rural = "gs21"
shed_midwest = "gs30;gs31"
shed_midwest_urban = "gs30"
shed_midwest_rural = "gs31"
shed_southwest = "gs40;gs41"
shed_southwest_urban = "gs40"
shed_southwest_rural = "gs41"
shed_west = "gs50;gs51"
shed_west_urban = "gs50"
shed_west_rural = "gs51"
shed_north = "gs60;gs61"
shed_north_urban = "gs60"
shed_north_rural = "gs61"
shed_niagara = "gs70;gs71"
shed_niagara_urban = "gs70"
shed_niagara_rural = "gs71"
shed_urban = "gs10;gs20;gs30;gs40;gs50;gs60;gs70"
shed_rural = "gs11;gs21;gs31;gs41;gs51;gs61;gs71"

shed_link_file = "C:\\Users\\mnasterska\\Documents\\Projects\\GGH\MOE\\Network_Links_Tagged_by_Shed_117.csv"
#assumes all auto centroid connectors have vdf = 90 exclusively

#auto occupancy
sov_oc = 1.2
hov2_oc = 2.2
hov3_oc = 3.3

#number of hours for transit assignment
num_hrs_am = 2
#num_hrs_md = 6

In [7]:
#2011 
#pop_o_matrix = "mo10"
#emp_o_matrix = "mo8"
#senior_o_matrix = "mo11"
#child_o_matrix = "mo7"
#low_inc_o_matrix = "mo9"
#youth_o_matrix = "mo12"
#pop_d_matrix = "md3"
#emp_d_matrix = "md4"

### 2011 Base
#sc_num_am = 101
#sc_num_md = 102
#scenario_folder = sc_2011


pop_o_matrix = "mo1"
emp_o_matrix = "mo2"
senior_o_matrix = "mo3"
child_o_matrix = "mo4"
low_inc_o_matrix = "mo5"# number of low inc households
youth_o_matrix = "mo6"
pop_d_matrix = "md1"
emp_d_matrix = "md2"

### 2041 Base
#sc_num_am = 119
#sc_num_md = 120
#scenario_folder = sc_2041_base

### 2041 DoNothing
#sc_num_am = 105
#sc_num_md = 106
#scenario_folder = sc_2041_donothing

### 2041 Committed+
#sc_num_am = 114
#sc_num_md = 115
#scenario_folder = sc_2041_C_plus_w_GTAWest_new

###2016
sc_num_am = 109
sc_num_md = 110
scenario_folder = sc_2016

#scenarios
sc_am = eb.scenario(sc_num_am)
sc_md = eb.scenario(sc_num_md)

#networks
ntwk_am = sc_am.get_network()
ntwk_md = sc_md.get_network()

#input folder
matrix_folder = u"P:\\451010\\281829_GGH\\GGHM Runs\\" + scenario_folder

In [9]:
#delete all FULL matrices
common.clear_matrices(eb)

In [4]:
print matrix_folder

P:\451010\281829_GGH\GGHM Runs\2041\MTO_2041_GGH_MTP_328_V105_CS222_20190522


In [10]:
#calculate min transit time from HDR toolbox transit time outputs
min_transit_time_peak = "mf20"

tt_dir = "M:\\10039241 - GGH Multimodal Transportation\\HDR_transit_time_results\\20190603_2041_Base_w_OL\\"
#tt_dir = "C:\\Users\\mnasterska\\Documents\\Projects\\GGH\\HDR_transit_time_results\\20181122_2041_plus\\"
tt_dir = "C:\\Users\\mnasterska\\Documents\\Projects\\GGH\\HDR_transit_time_results\\20181122_2016\\"

go_sub_time_mtx_array = ['1','3'] # matrix numbers for go train and subway time you want to include
common.minimum_transit_time(tt_dir,min_transit_time_peak,sc_am,go_sub_time_mtx_array)
    
common.clear_matrices(eb,[min_transit_time_peak],"FULL")

Imported matrix mf6
Imported matrix mf2
Imported matrix mf4
Finished Importing Matrices
Finished Calculating OD Transit Travel Times


In [5]:
#define matrix inputs

# demand
# peak period - 7 hrs
sov_demand_pk = "auto_sov_PkPd"
hov2_demand_pk = "auto_hov2_PkPd"
hov3_demand_pk = "auto_hov3_PkPd"
bike_demand_pk = "Bike_PkPd"
walk_demand_pk = "Walk_PkPd"
transit_demand_pk = "PnA_Transit_PkPd"
hbw_demand_pk = "trips_peak_all_modes_hbw_all_segments"

#am peak hour
sov_demand_am = "auto_sov_AMPk"
hov2_demand_am = "auto_hov2_AMPk"
hov3_demand_am = "auto_hov3_AMPk"
transit_2hr_demand_am = "total_transit_AMPk"
truck_l_demand_am = "lt_truck_ampkhr"
truck_m_demand_am = "mt_truck_ampkhr"
truck_h_demand_am = "ht_truck_ampkhr"

#offpeak
sov_demand_offpk = "auto_sov_offPkPd"
hov2_demand_offpk = "auto_hov2_offPkPd"
hov3_demand_offpk = "auto_hov3_offPkPd"
bike_demand_offpk = "Bike_offPkPd"
walk_demand_offpk = "Walk_offPkPd"
transit_demand_offpk = "PnA_Transit_OffPeakPd"

#midday peak hour?
sov_demand_md = "auto_sov_Midday"
hov2_demand_md = "auto_hov2_Midday"
hov3_demand_md = "auto_hov3_Midday"

# distance
sov_dist_am = "skim.am_peak.auto.sov.dist"

# travel time
active_time = "skim.static.bike.time"
sov_time_am = "skim.am_peak.auto.sov.time"
hov2_time_am = "skim.am_peak.auto.hov2.time"
hov3_time_am = "skim.am_peak.auto.hov3.time"
sov_time_md = "skim.midday.auto.sov.time"
hov2_time_md = "skim.midday.auto.hov2.time"
hov3_time_md = "skim.midday.auto.hov3.time"

#costs
sov_toll_am = "skim.am_peak.auto.sovt.toll_cost"
hov2_toll_am = "skim.am_peak.auto.hov2t.toll_cost"
hov3_toll_am = "skim.am_peak.auto.hov3t.toll_cost"
transit_fares_pk = "skim.peak.transit.go_bus.fares"




In [6]:
#2041 C+ and new runs

sov_demand_pk = "trips_peak_sov"
hov2_demand_pk = "trips_peak_hov2"
hov3_demand_pk = "trips_peak_hov3pl"
bike_demand_pk = "trips_peak_bike"
walk_demand_pk = "trips_peak_walk"
transit_demand_pk = "trips_transit_peak"
#HBW
hbw_demand_pk = "trips_peak_all_modes_hbw_all_segments" 
hbw_local_bus_demand_pk = "trips_peak_local_bus_HBW"
hbw_rapid_bus_demand_pk = "trips_peak_rapid_bus_HBW"
hbw_subway_demand_pk = "trips_peak_subway_HBW"
hbw_train_demand_pk = "trips_peak_train_HBW"
hbw_express_bus_demand_pk = "trips_peak_express_bus_HBW"

hbw_sov_demand_pk = "trips_peak_sov_HBW"
hbw_hov2_demand_pk = "trips_peak_hov2_HBW"
hbw_hov3pl_demand_pk = "trips_peak_hov3pl_HBW"

hbw_bike_demand_pk = "trips_peak_bike_HBW"
hbw_walk_demand_pk = "trips_peak_walk_HBW"
###
sov_demand_offpk = "trips_offpeak_sov"
hov2_demand_offpk = "trips_offpeak_hov2"
hov3_demand_offpk = "trips_offpeak_hov3pl"
bike_demand_offpk = "trips_offpeak_bike"
walk_demand_offpk = "trips_offpeak_walk"
transit_demand_offpk = "trips_transit_offpeak"
hov3_demand_am = "trips_am_peak_auto_hov3pl_pkhr"
sov_demand_am = "trips_am_peak_auto_sov_pkhr"
hov2_demand_am = "trips_am_peak_auto_hov2_pkhr"
transit_2hr_demand_am = "trips_transit_peak" #fix?

In [30]:
#MTP 328 re-naming

# peak period - 7 hrs
sov_demand_pk = "TRIPS SOV peak"
hov2_demand_pk = "TRIPS HOV2 peak"
hov3_demand_pk = "TRIPS HOV3 peak"
bike_demand_pk = "TRIPS BIKE peak"
walk_demand_pk = "TRIPS WALK peak"
transit_demand_pk = "TRIPS_TOTAL_TRANSIT_peak"

sov_demand_offpk = "TRIPS SOV offpeak"
hov2_demand_offpk = "TRIPS HOV2 offpeak"
hov3_demand_offpk = "TRIPS HOV3 offpeak"
bike_demand_offpk = "TRIPS BIKE offpeak"
walk_demand_offpk = "TRIPS WALK offpeak"
transit_demand_offpk = "TRIPS_TOTAL_TRANSIT_offpeak"

transit_2hr_demand_am = "TRIPS_TOTAL_TRANSIT_peak" #fix?

transit_fares_pk = "2041_fare_matrix_OLN"

#HBW
hbw_local_bus_drive_demand_pk = "trips.peak.hbw.transit.local_bus.drive"
hbw_local_bus_walk_demand_pk = "trips.peak.hbw.transit.local_bus.walk"
hbw_rapid_bus_drive_demand_pk = "trips.peak.hbw.transit.rapid_bus.drive"
hbw_rapid_bus_walk_demand_pk = "trips.peak.hbw.transit.rapid_bus.walk"
hbw_express_bus_demand_pk = "trips.peak.hbw.transit.express_bus"
hbw_express_bus_walk_demand_pk = "trips.peak.hbw.transit.express_bus.walk"

hbw_subway_knr_demand_pk = "od_trips.peak.hbw.transit.subway.drop_off"
hbw_subway_pnr_demand_pk = "od_trips.peak.hbw.transit.subway.park_access"
hbw_subway_tr_demand_pk = "od_trips.peak.hbw.transit.subway.transit_access"
hbw_subway_wk_demand_pk = "od_trips.peak.hbw.transit.subway.walk_access"
hbw_train_knr_demand_pk = "od_trips.peak.hbw.transit.train.drop_off"
hbw_train_pnr_demand_pk = "od_trips.peak.hbw.transit.train.park_access"
hbw_train_tr_demand_pk = "od_trips.peak.hbw.transit.train.transit_access"
hbw_train_wk_demand_pk = "od_trips.peak.hbw.transit.train.walk_access"

hbw_auto1_nt_demand_pk = "trips.peak.hbw.auto1.nt"
hbw_auto1_toll_demand_pk = "trips.peak.hbw.auto1.toll"
hbw_auto2_nt_hov_demand_pk = "trips.peak.hbw.auto2.nt_hov"
hbw_auto2_nt_nh_demand_pk = "trips.peak.hbw.auto2.nt_nh"
hbw_auto2_toll_hov_demand_pk = "trips.peak.hbw.auto2.toll_hov"
hbw_auto2_toll_nh_demand_pk = "trips.peak.hbw.auto2.toll_nh"
hbw_auto3_nt_hov_demand_pk = "trips.peak.hbw.auto3.nt_hov"
hbw_auto3_nt_nh_demand_pk = "trips.peak.hbw.auto3.nt_nh"
hbw_auto3_toll_hov_demand_pk = "trips.peak.hbw.auto3.toll_hov"
hbw_auto3_toll_nh_demand_pk = "trips.peak.hbw.auto3.toll_nh"

hbw_bike_demand_pk = "trips.peak.hbw.active.bike"
hbw_walk_demand_pk = "trips.peak.hbw.active.walk"

In [31]:
#matrices
common.clear_matrices(eb,[min_transit_time_peak],"FULL")
matrix_list = {
    sov_demand_pk : "",
    hov2_demand_pk : "",
    hov3_demand_pk : "",
    bike_demand_pk : "",
    walk_demand_pk : "",
    transit_demand_pk : "",
    hbw_local_bus_drive_demand_pk : "",
    hbw_local_bus_walk_demand_pk : "",
    hbw_rapid_bus_drive_demand_pk : "",
    hbw_rapid_bus_walk_demand_pk : "",
    hbw_express_bus_demand_pk : "",
    hbw_express_bus_walk_demand_pk : "",
    hbw_subway_knr_demand_pk : "",
    hbw_subway_pnr_demand_pk : "",
    hbw_subway_tr_demand_pk : "",
    hbw_subway_wk_demand_pk : "",
    hbw_train_knr_demand_pk : "",
    hbw_train_pnr_demand_pk : "",
    hbw_train_tr_demand_pk : "",
    hbw_train_wk_demand_pk : "",
    hbw_auto1_nt_demand_pk : "",
    hbw_auto1_toll_demand_pk : "",
    hbw_auto2_nt_hov_demand_pk : "",
    hbw_auto2_nt_nh_demand_pk : "",
    hbw_auto2_toll_hov_demand_pk : "",
    hbw_auto2_toll_nh_demand_pk : "",
    hbw_auto3_nt_hov_demand_pk : "",
    hbw_auto3_nt_nh_demand_pk: "",
    hbw_auto3_toll_hov_demand_pk : "",
    hbw_auto3_toll_nh_demand_pk : "",
    hbw_bike_demand_pk : "",
    hbw_walk_demand_pk : "",
    transit_2hr_demand_am : "",
    truck_l_demand_am : "",
    truck_m_demand_am : "",
    truck_h_demand_am : "",
    sov_demand_offpk : "",
    hov2_demand_offpk : "",
    hov3_demand_offpk : "",
    active_time : "",
    sov_dist_am : "",
    sov_time_am: "",
    hov2_time_am: "",
    hov3_time_am : "",
    #sov_time_md : "",
    #hov2_time_md: "",
    #hov3_time_md : "",
    bike_demand_offpk:"",
    walk_demand_offpk:"",
    transit_demand_offpk:"",
    sov_toll_am :"",
    hov2_toll_am :"",
    hov3_toll_am :"",
    transit_fares_pk :""
    } 

#Import matrices
for matrix_name in matrix_list.keys():
    matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
    print matrix_name
    create_matrix(matrix_id = matrix_list[matrix_name],
                  matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                  matrix_description = matrix_name)
    
common.import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",sc_am) 


trips.peak.hbw.transit.express_bus
TRIPS WALK offpeak
od_trips.peak.hbw.transit.train.park_access
TRIPS SOV peak
trips.peak.hbw.active.walk
trips.peak.hbw.auto2.toll_hov
trips.peak.hbw.auto2.nt_hov
od_trips.peak.hbw.transit.train.walk_access
trips.peak.hbw.auto3.nt_nh
trips.peak.hbw.active.bike
TRIPS HOV2 peak
TRIPS HOV2 offpeak
TRIPS_TOTAL_TRANSIT_peak
trips.peak.hbw.auto2.toll_nh
skim.am_peak.auto.sovt.toll_cost
mt_truck_ampkhr
ht_truck_ampkhr
skim.am_peak.auto.hov2t.toll_cost
trips.peak.hbw.auto3.toll_hov
od_trips.peak.hbw.transit.subway.transit_access
od_trips.peak.hbw.transit.train.drop_off
TRIPS HOV3 offpeak
TRIPS SOV offpeak
trips.peak.hbw.auto3.toll_nh
od_trips.peak.hbw.transit.subway.walk_access
od_trips.peak.hbw.transit.subway.park_access
TRIPS HOV3 peak
lt_truck_ampkhr
skim.am_peak.auto.hov3.time
TRIPS BIKE peak
trips.peak.hbw.transit.rapid_bus.drive
2041_fare_matrix_OLN
skim.static.bike.time
skim.am_peak.auto.hov2.time
trips.peak.hbw.transit.local_bus.walk
TRIPS BIKE offpea

In [55]:
#add matrices for total trips for RWDI moes

matrix_list = {
    sov_demand_pk : "",
    hov2_demand_pk : "",
    hov3_demand_pk : "",
    sov_demand_am : "",
    hov2_demand_am : "",
    hov3_demand_am : "",
    sov_demand_offpk : "",
    hov2_demand_offpk : "",
    hov3_demand_offpk : "",
    sov_demand_md : "",
    hov2_demand_md : "",
    hov3_demand_md : ""
    } 


In [4]:
#for mode split plots 
matrix_folder = u"P:\\451010\\281829_GGH\\GGHM Runs\\2041\\GGHM Base Case\\USE - Sent Through FTP September 20 2018"

matrix_list ={
    "trips_all_modes_daily" :"",
    "trips_auto_daily" :"",
    "trips_transit_daily" :"",
    "trips_active_daily" :""
    }

In [41]:
matrix_list ={
    "trips_am_peak_auto1_nt" :"",
    "trips_am_peak_auto1_toll" :"",
    "trips_am_peak_auto2_nt_hov" :"",
    "trips_am_peak_auto2_nt_nh" :"",
    "trips_am_peak_auto2_toll_hov" :"",
    "trips_am_peak_auto2_toll_nh" :"",
    "trips_am_peak_auto3_nt_hov" :"",
    "trips_am_peak_auto3_nt_nh" :"",
    "trips_am_peak_auto3_toll_hov" :"",
    "trips_am_peak_auto3_toll_nh" :"",
    "trips_midday_auto1_nt" :"",
    "trips_midday_auto1_toll" :"",
    "trips_midday_auto2_nt_hov" :"",
    "trips_midday_auto2_nt_nh" :"",
    "trips_midday_auto2_toll_hov" :"",
    "trips_midday_auto2_toll_nh" :"",
    "trips_midday_auto3_nt_hov" :"",
    "trips_midday_auto3_nt_nh" :"",
    "trips_midday_auto3_toll_hov" :"",
    "trips_midday_auto3_toll_nh" :""
    }

In [28]:
#Import matrices
for matrix_name in matrix_list.keys():
    matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = matrix_list[matrix_name],
                  matrix_name= matrix_name.replace(" ","_"),
                  matrix_description = matrix_name)

In [43]:
common.import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",sc_am) 

Imported matrix mf59
Imported matrix mf50
Imported matrix mf42
Imported matrix mf64
Imported matrix mf40
Imported matrix mf43
Imported matrix mf60
Imported matrix mf38
Imported matrix mf39
Imported matrix mf48
Imported matrix mf67
Imported matrix mf41
Imported matrix mf53
Imported matrix mf68
Imported matrix mf56
Imported matrix mf66
Imported matrix mf55
Imported matrix mf69
Imported matrix mf45
Imported matrix mf52
ht_truck_ampkhr
Imported matrix mf57
lt_truck_ampkhr
Imported matrix mf46
mt_truck_ampkhr
Imported matrix mf54
Matrix skim.peak.transit.go_bus.fares was not imported, or contains no values!
Matrix trips_peak_hov3pl_HBW was not imported, or contains no values!
Matrix trips_peak_train_HBW was not imported, or contains no values!
Matrix trips_peak_bike_HBW was not imported, or contains no values!
Matrix trips_peak_express_bus_HBW was not imported, or contains no values!
Matrix trips_peak_walk_HBW was not imported, or contains no values!
Matrix trips_peak_hov2_HBW was not impor

In [7]:
common.clear_matrices(eb,["mo1","mo2","mo3","mo4","mo5","mo6","mo11"],"ORIGIN")

In [8]:
emp_auto  = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = emp_auto,
                  matrix_name= "employment.auto.demand",
                  matrix_description = "employment.auto.demand")

emp_total = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = emp_total,
                  matrix_name= "employment.total.demand",
                  matrix_description = "employment.total.demand")

emp_transit = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = emp_transit,
                  matrix_name= "employment.transit.demand",
                  matrix_description = "employment.transit.demand")

emp_active = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = emp_active,
                  matrix_name= "employment.active.demand",
                  matrix_description = "employment.active.demand")


OriginMatrix(mo10)

In [10]:
#calculate proportion of auto trips to employment area

emp_zones = 'ge19'

#st catharines UGC
#emp_zones = '7114;7115;7120;7121;7124;7131'

#Barrie UGC
emp_zones = '8845;8848;8853;8856'

#Newmarket UGC
emp_zones = '3412;3413;3421'

#Pickering UGC
emp_zones = '2015;2016;2019'

#Oshawa UGC
emp_zones = '2132;2136;2140'

#Brantford UGC
emp_zones = '8117;8118;8120;8123;8128'

#Guelph UGC
emp_zones = '8542;8545;8547;8548;8550;8551;8554;8557;8558;8559;8561;8563'

#Hamilton UGC
emp_zones = '6169;6173;6186;6188;6189;6192;6195;6197;6201'

spec = common.get_mc_spec(emp_total, matrix_list['trips_all_modes_daily'],None,"+","all", emp_zones ,None)
matrix_calculator(spec)
spec = common.get_mc_spec(emp_auto, matrix_list['trips_auto_daily'],None,"+","all", emp_zones ,None)
matrix_calculator(spec)
spec = common.get_mc_spec(emp_transit, matrix_list['trips_transit_daily'],None,"+","all", emp_zones ,None)
matrix_calculator(spec)
spec = common.get_mc_spec(emp_active, matrix_list['trips_active_daily'],None,"+","all", emp_zones ,None)
matrix_calculator(spec)

{'average': 0.30205,
 'expression_type': 'FULL',
 'maximum': 391.751922,
 'maximum_at': {'destination': 6188, 'origin': 6188},
 'minimum': 0.0,
 'minimum_at': {'destination': 6169, 'origin': 1001},
 'num_evaluations': 31356,
 'result_type': 'ORIGIN',
 'sum': 9471.077148}

In [22]:
eb.delete_matrix(emp_auto)
eb.delete_matrix(emp_transit)
eb.delete_matrix(emp_active)
eb.delete_matrix(emp_total)

In [36]:
#aggregate total truck trip demand for plots
truck_am = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = truck_am,
                  matrix_name= "truck.demand.am",
                  matrix_description = "truck.demand.am")
truck_md = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id =  truck_md,
                  matrix_name= "truck.demand.md",
                  matrix_description = "truck.demand.md")

OriginMatrix(mo13)

In [10]:
#calculate proportion of auto trips to employment area

truck_zones = '6187'

spec = common.get_mc_spec(truck_am, "mf95+mf90+mf85" ,None,"+","all", truck_zones ,None)
matrix_calculator(spec)
spec = common.get_mc_spec(truck_md, "mf96+mf91+mf86" ,None,"+","all", truck_zones ,None)
matrix_calculator(spec)

{'average': 0.175487,
 'expression_type': 'FULL',
 'maximum': 12.868502,
 'maximum_at': {'destination': 6187, 'origin': 5051},
 'minimum': 0.0,
 'minimum_at': {'destination': 6187, 'origin': 1001},
 'num_evaluations': 3484,
 'result_type': 'ORIGIN',
 'sum': 611.396545}

In [22]:
print matrix_list

{'trips_auto_daily': 'mf1', 'trips_active_daily': 'mf2', 'trips_transit_daily': 'mf3', 'trips_all_modes_daily': 'mf4'}


In [37]:
#total trips for rwdi

exp_pk = matrix_list[sov_demand_pk] + "+" + matrix_list[hov2_demand_pk] + "+" + matrix_list[hov3_demand_pk]
exp_am = matrix_list[sov_demand_am] + "+" + matrix_list[hov2_demand_am] + "+" + matrix_list[hov3_demand_am]
exp_offpk = matrix_list[sov_demand_offpk] + "+" + matrix_list[hov2_demand_offpk] + "+" + matrix_list[hov3_demand_offpk]
exp_md = matrix_list[sov_demand_md] + "+" + matrix_list[hov2_demand_md] + "+" + matrix_list[hov3_demand_md]

spec = common.get_mc_spec(None, exp_pk,"+","+","all","all",None)
print matrix_calculator(spec)['result']

15907628.0


In [9]:
#2041 committed +

exp_am = matrix_list["trips_am_peak_auto1_nt"] + "+" + matrix_list["trips_am_peak_auto1_toll"]\
    + "+" + matrix_list["trips_am_peak_auto2_nt_hov"] + "+" + matrix_list["trips_am_peak_auto2_nt_nh"]\
    + "+" + matrix_list["trips_am_peak_auto2_toll_hov"] + "+" + matrix_list["trips_am_peak_auto2_toll_nh"]\
    + "+" + matrix_list["trips_am_peak_auto3_nt_hov"] + "+" + matrix_list["trips_am_peak_auto3_nt_nh"]\
    + "+" + matrix_list["trips_am_peak_auto3_toll_hov"] + "+" + matrix_list["trips_am_peak_auto3_toll_nh"]

exp_md = matrix_list["trips_midday_auto1_nt"] + "+" + matrix_list["trips_midday_auto1_toll"]\
    + "+" + matrix_list["trips_midday_auto2_nt_hov"] + "+" + matrix_list["trips_midday_auto2_nt_nh"]\
    + "+" + matrix_list["trips_midday_auto2_toll_hov"] + "+" + matrix_list["trips_midday_auto2_toll_nh"]\
    + "+" + matrix_list["trips_midday_auto3_nt_hov"] + "+" + matrix_list["trips_midday_auto3_nt_nh"]\
    + "+" + matrix_list["trips_midday_auto3_toll_hov"] + "+" + matrix_list["trips_midday_auto3_toll_nh"]

spec = common.get_mc_spec(None, exp_md,"+","+","all","all",None)
print matrix_calculator(spec)['result']

KeyError: 'trips_am_peak_auto1_nt'

In [32]:
#Expressions

#total person trips - Peak Period,
exp_pk_pers_trips = "(" + matrix_list[hov2_demand_pk] + "*"+ str(hov2_oc)\
    + "+" + matrix_list[hov3_demand_pk] + "*" + str(hov3_oc)\
    + "+" + matrix_list[sov_demand_pk] + "*" + str(sov_oc)\
    + "+" + matrix_list[bike_demand_pk] + "+" + matrix_list[transit_demand_pk] \
    + "+" + matrix_list[walk_demand_pk] + ")"
    
exp_offpk_pers_trips = "(" + matrix_list[hov2_demand_offpk] + "*"+ str(hov2_oc)\
    + "+" + matrix_list[hov3_demand_offpk] + "*" + str(hov3_oc)\
    + "+" + matrix_list[sov_demand_offpk] + "*" + str(sov_oc)\
    + "+" + matrix_list[bike_demand_offpk] + "+" + matrix_list[transit_demand_offpk] \
    + "+" + matrix_list[walk_demand_offpk] + ")"

exp_pk_pers_trip_time = "(" + matrix_list[hov2_demand_pk] + "*"+ str(hov2_oc) + "*" + matrix_list[hov2_time_am]\
    + "+" + matrix_list[hov3_demand_pk] + "*" + str(hov3_oc) + "*" + matrix_list[hov3_time_am]\
    + "+" + matrix_list[sov_demand_pk] + "*" + str(sov_oc) + "*" + matrix_list[sov_time_am]\
    + "+" + matrix_list[bike_demand_pk] + "*" + matrix_list[active_time]\
    + "+" + matrix_list[transit_demand_pk] + "*" + min_transit_time_peak\
    + "+" + matrix_list[walk_demand_pk] + "*" + matrix_list[active_time] + "*5)"
    
exp_pk_pers_trip_speed = "(60*" + matrix_list[sov_dist_am]\
    + "*(" + matrix_list[hov2_demand_pk] + "*"+ str(hov2_oc)+ "/" + matrix_list[hov2_time_am]+ "*(" + matrix_list[hov2_time_am] + "!=0)+"\
    + matrix_list[hov3_demand_pk] + "*" + str(hov3_oc) + "/" + matrix_list[hov3_time_am]+ "*(" + matrix_list[hov3_time_am] + "!=0)+"\
    + matrix_list[sov_demand_pk] + "*" + str(sov_oc) + "/" + matrix_list[sov_time_am]+ "*(" + matrix_list[sov_time_am] + "!=0)+"\
    + matrix_list[transit_demand_pk] + "/" + min_transit_time_peak + "*(" + min_transit_time_peak + "!=0)))"

exp_pk_pers_trips_no_active = "(" + matrix_list[hov2_demand_pk] + "*"+ str(hov2_oc)\
    + "+" + matrix_list[hov3_demand_pk] + "*" + str(hov3_oc)\
    + "+" + matrix_list[sov_demand_pk] + "*" + str(sov_oc)\
    + "+" + matrix_list[transit_demand_pk] + ")"
    
exp_pk_min_travel_time = "(" + min_transit_time_peak + ".min." + matrix_list[active_time] \
+ ".min." + matrix_list[hov3_time_am] + ")"

exp_pk_trk_trips = matrix_list[truck_l_demand_am] + "+" + matrix_list[truck_m_demand_am]\
+ "+" + matrix_list[truck_h_demand_am]

exp_pk_trk_speed = "(60*" + matrix_list[sov_dist_am]+ "/" + matrix_list[sov_time_am]\
    + "*(" + matrix_list[truck_l_demand_am] + "+" + matrix_list[truck_m_demand_am]\
+ "+" + matrix_list[truck_h_demand_am] + ")*(" + matrix_list[sov_time_am] + "!=0))"

In [33]:
#functions

def MOE_fraction(out_file,numerator,denominator,zones_array):
    
    for origins in zones_array:
        spec = common.get_mc_spec(None, denominator,"+","+",origins,"all",None)
        bottom = matrix_calculator(spec)['result']
        spec = common.get_mc_spec(None, numerator,"+","+",origins,"all",None)
        top = float(matrix_calculator(spec)['result'])
        out_file.write("{0:.4f},".format(top/bottom))
    out_file.write("\n")

In [34]:
zones_array = ["all",zones_urban, zones_rural,zones_pd1,\
               shed_central_urban,shed_central_rural,shed_central,\
               shed_east_urban,shed_east_rural,shed_east,\
               shed_midwest_urban,shed_midwest_rural,shed_midwest,\
               shed_southwest_urban,shed_southwest_rural,shed_southwest,\
               shed_west_urban,shed_west_rural,shed_west,\
               shed_north_urban,shed_north_rural,shed_north,\
               shed_niagara_urban,shed_niagara_rural,shed_niagara]

In [51]:
out_file = "C:\\Users\\mnasterska\\Documents\\Projects\\GGH\\MOE\\20190605_2041.csv"

moef = open(out_file, 'w')
moef.write("MOE," + ",".join(zones_array) + "\n")


In [147]:
#2019-02-26 average trip length 

spec = common.get_mc_spec(None, exp_pk_pers_trips,"+","+","all",shed_niagara,None)
all_trips = matrix_calculator(spec)['result']

spec['constraint']['by_value'] = {}
spec['constraint']['by_value']['od_values'] = matrix_list[sov_dist_am]
spec['constraint']['by_value']['interval_min'] = 0
spec['constraint']['by_value']['condition'] = 'INCLUDE'

In [151]:
dist_dist = {}

for dist in range(1,50):
    
    spec['constraint']['by_zone']['destinations'] = zones_ugc
    
    spec['constraint']['by_value']['interval_min'] = dist-1
    spec['constraint']['by_value']['interval_max'] = dist
    
    spec['expression'] = exp_pk_pers_trips
    all_modes = matrix_calculator(spec)['result']
    
    spec['constraint']['by_zone']['destinations'] = zones_pd1
    all_modes -= matrix_calculator(spec)['result']
    #print " all " , all_modes
    
    spec['constraint']['by_zone']['destinations'] = zones_ugc
    #spec['expression'] = matrix_list[transit_demand_pk] 
    spec['expression'] = matrix_list[bike_demand_pk] + "+" + matrix_list[walk_demand_pk]
    
    only_transit = matrix_calculator(spec)['result']
    spec['constraint']['by_zone']['destinations'] = zones_pd1
    only_transit -= matrix_calculator(spec)['result']
    #print " transit " , only_transit
    
    print only_transit/all_modes

0.11121212717
0.0632152681288
0.0466313309125
0.0340837139022
0.0253055552179
0.0190006409521
0.0144355853834
0.0104855766225
0.00818917389656
0.00629421688757
0.00468350012852
0.00326165340229
0.00238042937902
0.00182269980507
0.00141124562537
0.000960798950318
0.000854887432207
0.000605975559101
0.000410790127252
0.000341320919433
0.000270652802727
0.00024284445521
0.00019806040993
0.000151470431916
0.000145536569095
9.46806410145e-05
5.98987176507e-05
6.09057071688e-05
3.58106252417e-05
3.7640947605e-05
2.99080139248e-05
2.27162636238e-05
1.18502559088e-05
8.73595649116e-06
1.15502405733e-05
8.88511524614e-06
8.92689073079e-06
3.98976611716e-06
3.18444728594e-06
3.17009948157e-06
2.4236878877e-06
2.38490106944e-06
8.55299321352e-07
7.81535225023e-07
6.97050754223e-07
4.12849447568e-07
7.46559296814e-07
2.2267371237e-07
4.37066272781e-07


In [ ]:
#distance vs. mode share

for dist in range(1,20):
    spec['constraint']['by_value']['interval_min'] = dist-1
    spec['constraint']['by_value']['interval_max'] = dist
    dist_dist[dist] = matrix_calculator(spec)['result']/all_trips

    print dist_dist[dist]

In [50]:
print exp_pk_pers_trips + "*" + matrix_list[sov_dist_am]

(mf4*2.2+mf6*3.3+mf23*1.2+mf2+mf1+mf19)*mf15


In [52]:
# 2018-07-26 gap analysis
#"01	Connected	"
#"	01	People and goods are able to move efficiently with minimal delay 	"

moef.write("01	Average travel speed for people( auto and transit only),")

MOE_fraction(moef,exp_pk_pers_trip_speed,exp_pk_pers_trips_no_active,zones_array)


moef.write("Average travel speed for goods,")


MOE_fraction(moef,exp_pk_trk_speed,exp_pk_trk_trips,zones_array)

#print "04	A well connected road and highway network supports the movement of people and goods throughout the region"
moef.write("% of people and jobs in the GGH accessible on the road network (no transit) within 45 min,")

expression_access = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + matrix_list[hov3_time_am]  + " <45)"
expression_total = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")"


MOE_fraction(moef,expression_access,expression_total,zones_array)

#print "04	A well connected road and highway network supports the movement of people and goods throughout the region"
moef.write("# of people and jobs in the GGH accessible on the road network (no transit) within 45 min,")

expression_access = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + matrix_list[hov3_time_am]  + " <45)"
expression_total = pop_o_matrix


for origins in zones_array:
    spec = common.get_mc_spec(None, expression_access,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_total,"+",None ,origins,None,None)
    bottom = float(matrix_calculator(spec)['result'])
    moef.write("{0:.4f},".format(top/bottom))
moef.write("\n")

moef.write("02	% of people and jobs accessible from goods movement centres (including themselves) 	,")
           
expression_access = emp_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + matrix_list[hov3_time_am]  + " <45)"
expression_total = emp_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")"

zones_gmc = "4182;4380;4216;4381;4378;1018;6019;8370;3209;2190;\
            2075;1054;1074;1082;1087;1115;6199;6209;6211-6212;5081;\
            4292;3078;5171"

#"Aggregate employment lands (weighted average based on number of jobs in that employment),"
for destination in zones_array:
    
    spec = common.get_mc_spec(None, expression_access,"+","+",zones_gmc,destination,None)
    access = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_total,"+","+",zones_gmc,destination,None)
    total = matrix_calculator(spec)['result']
    moef.write("{0:.2f}%,".format(access/total*100))
moef.write("\n")

#"02	Integrated	"
#"01	People live in complete communities where access to services, jobs and recreation is close and convenient"
moef.write("01	Person kilometres travelled (PKT) per capita,") 

expression  = "(" + matrix_list[hov2_demand_pk] + "*"+ str(hov2_oc)\
    + "+" + matrix_list[hov3_demand_pk] + "*" + str(hov3_oc)\
    + "+" + matrix_list[sov_demand_pk] + "*" + str(sov_oc)\
    + "+" + matrix_list[bike_demand_pk] + "+" + matrix_list[transit_demand_pk] \
    + "+" + matrix_list[walk_demand_pk] + ")*" + matrix_list[sov_dist_am]
    
for origins in zones_array:
    spec = common.get_mc_spec(None, pop_o_matrix,"+",None,origins,None,None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    moef.write( "{0:.2f},".format(top/bottom))
    
moef.write("\n")

#"03	Equitable	"
#"01	Reliable and frequent transit services can be accessed by all users regardless of age, income, or ability."
moef.write("01	Percent of jobs accessible by transit for low income,")

expression_access = low_inc_o_matrix + "*" + emp_d_matrix + "*(" + min_transit_time_peak  + " <45)"
expression_total = low_inc_o_matrix + "*" + emp_d_matrix 

MOE_fraction(moef,expression_access,expression_total,zones_array)

moef.write("01	Number of jobs accessible by transit for low income,")

expression_access = low_inc_o_matrix + "*" + emp_d_matrix + "*(" + min_transit_time_peak  + " <45)"
expression_total = low_inc_o_matrix 

for origins in zones_array:
    spec = common.get_mc_spec(None, expression_access,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_total,"+",None ,origins,None,None)
    bottom = float(matrix_calculator(spec)['result'])
    moef.write("{0:.4f},".format(top/bottom))
moef.write("\n")

moef.write("02	Percent of people accessible by transit for youth 	,")
expression_access = youth_o_matrix + "*(" + pop_d_matrix + ")*(" + min_transit_time_peak  + " <45)"
expression_total = youth_o_matrix + "*" + pop_d_matrix

MOE_fraction(moef,expression_access,expression_total,zones_array)

moef.write("02	Number of people accessible by transit for youth 	,")
expression_access = youth_o_matrix + "*(" + pop_d_matrix + ")*(" + min_transit_time_peak  + " <45)"
expression_total = youth_o_matrix

for origins in zones_array:
    spec = common.get_mc_spec(None, expression_access,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_total,"+",None ,origins,None,None)
    bottom = float(matrix_calculator(spec)['result'])
    moef.write("{0:.4f},".format(top/bottom))
moef.write("\n")

moef.write("03	Percent of people accessible by transit for seniors 	,")
expression_access = senior_o_matrix + "*" + pop_d_matrix + "*(" + min_transit_time_peak  + " <45)"
expression_total = senior_o_matrix + "*" + pop_d_matrix

MOE_fraction(moef,expression_access,expression_total,zones_array)

moef.write("03	Number of people accessible by transit for seniors 	,")
expression_access = senior_o_matrix + "*" + pop_d_matrix + "*(" + min_transit_time_peak  + " <45)"
expression_total = senior_o_matrix

for origins in zones_array:
    spec = common.get_mc_spec(None, expression_access,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_total,"+",None ,origins,None,None)
    bottom = float(matrix_calculator(spec)['result'])
    moef.write("{0:.4f},".format(top/bottom))
moef.write("\n")

moef.write("04	Percent of jobs accessible for general population 	,")
expression_access = pop_o_matrix + "*" + emp_d_matrix + "*(" + min_transit_time_peak  + " <45)"
expression_total = pop_o_matrix + "*" + emp_d_matrix

MOE_fraction(moef,expression_access,expression_total,zones_array)

moef.write("04	Number of jobs accessible for general population 	,")
expression_access = pop_o_matrix + "*" + emp_d_matrix + "*(" + min_transit_time_peak  + " <45)"
expression_total = pop_o_matrix

for origins in zones_array:
    spec = common.get_mc_spec(None, expression_access,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_total,"+",None ,origins,None,None)
    bottom = float(matrix_calculator(spec)['result'])
    moef.write("{0:.4f},".format(top/bottom))
moef.write("\n")

moef.write("user cost - all trips,")

exp_cost = matrix_list[sov_toll_am] + "/100*" + matrix_list[sov_demand_pk]\
    + "+" + matrix_list[hov2_toll_am] + "/100*" + matrix_list[hov2_demand_pk]\
    + "+" + matrix_list[hov3_toll_am] + "/100*" + matrix_list[hov3_demand_pk]\
    + "+" + matrix_list[transit_fares_pk] + "*" + matrix_list[transit_demand_pk]\
    + "+(" + matrix_list[sov_demand_pk] + "+" + matrix_list[hov2_demand_pk] + "+" + matrix_list[hov3_demand_pk]\
    + ")*" + matrix_list[sov_dist_am] + "*0.55"

exp_trips = matrix_list[sov_demand_pk] + "*" + str(sov_oc)\
    + "+" +  matrix_list[hov2_demand_pk] + "*" + str(hov2_oc)\
    + "+" + matrix_list[hov3_demand_pk] + "*" + str(hov3_oc)\
    + "+" + matrix_list[transit_demand_pk]

MOE_fraction(moef,exp_cost, exp_trips ,zones_array)

moef.write("02	PKT demand/PKT capacity for all motorized modes,")
#"(for private modes, capacity will be derived from vehicle lane capacity multiplied by auto occupancy)"
#"GGH Wide"
total_cap_lane_km = 0.0
total_person_lane_km = 0.0

for link in ntwk_am.links():
    #ignore centroid connectors
    if link.volume_delay_func != 90:
        #auto persons
        link_persons = (link["@volau1_nt"] + link["@volau1_toll"])*sov_oc \
        + ( link["@volau2_nt_hov"] + link["@volau2_nt_nh"] + link["@volau2_toll_hov"] + link["@volau2_toll_nh"])*hov2_oc \
        + ( link["@volau3_nt_hov"] + link["@volau3_nt_nh"] + link["@volau3_toll_hov"] + link["@volau3_toll_nh"])*hov3_oc
        
        #check if hov
        if link['@is_hov2'] == 0 and link['@is_hov3'] == 0:
            link_cap = link.data3*sov_oc*link.num_lanes
        else:
            link_cap = link.data3*(link['@is_hov2']*hov2_oc + link['@is_hov3']*hov3_oc)*link.num_lanes
        
        #transit
        for segment in link.segments():
            transit_line = segment.line
            link_cap += 60 * num_hrs_am / transit_line.headway * transit_line.vehicle.total_capacity
            link_persons += segment.transit_volume                
        if link_persons >0:
            total_cap_lane_km += link_cap*link.length
            total_person_lane_km += link_persons*link.length*link.num_lanes
moef.write(  "{0:.2f}\n".format(total_person_lane_km/total_cap_lane_km))








In [37]:
#"06	Active, Safe and Healthy	"
#"01	The transportation system supports walking and cycling, reducing dependence on personal vehicles	"
#moef.write( "01	PKT involving active transportation (walking and cycling)	,")
#moef.write( "01	PKT per capita involving active transportation (walking and cycling)	,")

expression_active = matrix_list[sov_dist_am] + "*(" \
+ matrix_list[walk_demand_pk] + "+" + matrix_list[bike_demand_pk] + ")"

temp_PKT = 0.0
temp_pop = 0.0

for origins in zones_array:
    spec = common.get_mc_spec(None, expression_active,"+","+",origins,"all",None)
    temp_PKT = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, pop_o_matrix,"+",None,origins,None,None)
    temp_pop = matrix_calculator(spec)['result']
    
    print (origins)
    print ( "Total PKT: {0:.0f},".format(temp_PKT))
    #print ( "Total population: {0:.0f},".format(temp_pop))
    print ( "PKT Per Captia: {0:.3f},".format(temp_PKT/temp_pop))  




all
Total PKT: 1860459,
PKT Per Captia: 0.138,
gb2;gb3;gb4
Total PKT: 1623529,
PKT Per Captia: 0.151,
gb1
Total PKT: 236929,
PKT Per Captia: 0.086,
gf1
Total PKT: 201222,
PKT Per Captia: 0.452,
gs10
Total PKT: 636855,
PKT Per Captia: 0.153,
gs11
Total PKT: 16449,
PKT Per Captia: 0.130,
gs10;gs11
Total PKT: 653304,
PKT Per Captia: 0.152,
gs20
Total PKT: 133914,
PKT Per Captia: 0.138,
gs21
Total PKT: 58014,
PKT Per Captia: 0.094,
gs20;gs21
Total PKT: 191928,
PKT Per Captia: 0.121,
gs30
Total PKT: 283662,
PKT Per Captia: 0.117,
gs31
Total PKT: 50180,
PKT Per Captia: 0.094,
gs30;gs31
Total PKT: 333842,
PKT Per Captia: 0.113,
gs40
Total PKT: 107942,
PKT Per Captia: 0.144,
gs41
Total PKT: 28242,
PKT Per Captia: 0.090,
gs40;gs41
Total PKT: 136183,
PKT Per Captia: 0.128,
gs50
Total PKT: 127178,
PKT Per Captia: 0.139,
gs51
Total PKT: 16637,
PKT Per Captia: 0.064,
gs50;gs51
Total PKT: 143815,
PKT Per Captia: 0.123,
gs60
Total PKT: 77418,
PKT Per Captia: 0.120,
gs61
Total PKT: 49953,
PKT Per Capt

In [53]:
#"08	Prosperous	"
# "01	The transportation system supports business and trade by ensuring efficient travel times to, from "
#print "and through gateways to the region (ports, airports, border crossings)	"
moef.write( "01	Average truck travel times to and from intermodal hubs/ports/airports/ and gateways,")

zones = "4182;4380;4216;4381;4378;1018;6019;8370;3209;2190;2075;1054;1074;1082;1087;1115;6199;6209;6211-6212;5081;4292;3078;5171;9400-9504"

exp_pk_trk_time = "(" + matrix_list[sov_time_am] + "*(" + matrix_list[truck_h_demand_am]\
    + "+" + matrix_list[truck_m_demand_am]\
    + "+" + matrix_list[truck_l_demand_am] + "))"

spec = common.get_mc_spec(None, exp_pk_trk_trips,"+","+",zones,zones,None)
bottom = matrix_calculator(spec)['result']
spec = common.get_mc_spec(None, exp_pk_trk_time ,"+","+",zones,zones,None)
top = matrix_calculator(spec)['result']
moef.write( "{0:.2f}\n".format(top/bottom))

# "03	Major employment areas are supported by a full range of transportation options	" 

moef.write("01	 Average travel time to nearest major employment areas from place of residence,")

min_dist = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = min_dist,
    matrix_name= "min.dist",
    matrix_description = "min.dist")
closest_map = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = closest_map,
    matrix_name= "closest.map",
    matrix_description = "closest.map")

spec = common.get_mc_spec(min_dist, matrix_list[sov_dist_am],None,".min.","all",zones_ugc + ";" + zones_employment,None)
matrix_calculator(spec)

spec = common.get_mc_spec(closest_map, min_dist + "==" + min_dist,None,None,"all",zones_ugc + ";" + zones_employment,None)
matrix_calculator(spec)

expression_time = "(" + closest_map + "*(" + min_transit_time_peak + ".min." + matrix_list[sov_time_am] + "))*"\
    + pop_o_matrix
expression_total = pop_o_matrix + "*" + closest_map
for origins in zones_array:
    spec = common.get_mc_spec(None, expression_total,"+","+",origins,"all",None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_time,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    moef.write("{0:.2f},".format(top/bottom))
moef.write("\n")
eb.delete_matrix(min_dist)
eb.delete_matrix(closest_map)

moef.write(" % of transit network congested - peak,")
cong_frac = 0.85
total_transit_pkt = 0.0
cong_transit_pkt = 0.0

for line in ntwk_am.transit_lines():
    for segment in line.segments():
        total_transit_pkt += segment.link.length * segment.transit_volume
        if segment.transit_volume/line.vehicle.total_capacity > cong_frac:
            cong_transit_pkt += segment.link.length * segment.transit_volume


moef.write( "{0:.2f}\n".format(cong_transit_pkt/total_transit_pkt))

moef.write(" % of transit network congested - offpeak,")

total_transit_pkt = 0.0
cong_transit_pkt = 0.0

for line in ntwk_md.transit_lines():
    for segment in line.segments():
        total_transit_pkt += segment.link.length * segment.transit_volume
        if segment.transit_volume/line.vehicle.total_capacity > cong_frac:
            cong_transit_pkt += segment.link.length * segment.transit_volume
    
moef.write( "{0:.2f}\n".format(cong_transit_pkt/total_transit_pkt))



In [54]:
#"04	Congestion is reduced, enabling people to accomplish more because they spend less time on travel	"
moef.write( "01	% of peak road network congested 	\n")

link_array = ["all",shed_urban, shed_rural,\
               shed_central_urban,shed_central_rural,shed_central,\
               shed_east_urban,shed_east_rural,shed_east,\
               shed_midwest_urban,shed_midwest_rural,shed_midwest,\
               shed_southwest_urban,shed_southwest_rural,shed_southwest,\
               shed_west_urban,shed_west_rural,shed_west,\
               shed_north_urban,shed_north_rural,shed_north,\
               shed_niagara_urban,shed_niagara_rural,shed_niagara]

cong_frac =0.85

total_auto_pkt = 0.0
total_truck_pkt = 0.0

cong_auto_pkt = 0.0
cong_truck_pkt = 0.0

veh_attributes = ["@volau1_nt","@volau1_toll",
                  "@volau2_nt_hov","@volau2_nt_nh","@volau2_toll_hov","@volau2_toll_nh",
                  "@volau3_nt_hov","@volau3_nt_nh","@volau3_toll_hov","@volau3_toll_nh"]

truck_attributes = ["@volau_htruck","@volau_ltruck","@volau_mtruck"]

#get link sheds

link_tags = {}
vkt_veh_shed = {}
vkt_cong_veh_shed= {}

with open(shed_link_file, "r") as tag_file:
    for row in tag_file.readlines()[1:]:
        row = row.strip("\n")
        currow = row.split("\t")
        i_node = currow[4]
        j_node = currow[5]
        shed = int(float(currow[3]))
        link_tags[i_node + "-" + j_node] = shed              

    
for link in ntwk_am.links():
    #ignore centroid connectors, transfer links
    
    if link.length > 0 and link.data3!= 9999 and link.auto_volume >0:
        link_veh = 0.0
        link_truck = 0.0
        
        if link.id in link_tags:
            shed = "gs" + str(link_tags[link.id])
        else:
            shed = None
            
        for volume in veh_attributes:
            link_veh += link[volume]
        for volume in truck_attributes:
            link_truck += link[volume]
        total_auto_pkt += link_veh*link.length
        total_truck_pkt += link_truck*link.length
        
        if shed:
            if vkt_veh_shed.has_key(shed):
                vkt_veh_shed[shed] += link_veh*link.length
            else:
                vkt_veh_shed[shed] = link_veh*link.length            
        if vkt_veh_shed.has_key("all"):
            vkt_veh_shed["all"] += link_veh*link.length
        else:
            vkt_veh_shed["all"] = link_veh*link.length

        try:
            if (link_veh+link_truck)/(link.num_lanes*link.data3) > cong_frac:
                cong_auto_pkt += link_veh*link.length
                cong_truck_pkt += link_truck*link.length
                
                if shed:
                    if vkt_cong_veh_shed.has_key(shed):
                        vkt_cong_veh_shed[shed] += link_veh*link.length
                    else:
                        vkt_cong_veh_shed[shed] = link_veh*link.length                    
                if vkt_cong_veh_shed.has_key("all"):
                    vkt_cong_veh_shed["all"] += link_veh*link.length
                else:
                    vkt_cong_veh_shed["all"] = link_veh*link.length

                
        except:
            print link.id
            
moef.write( "Vehicle,")
#moef.write( "{0:.2f}%\n".format(cong_auto_pkt/total_auto_pkt*100))
for zone_group in link_array:
    zone_list = zone_group.split(";")
    cong_vkt = 0.0
    total_vkt = 0.0
    for zone in zone_list:
        cong_vkt += vkt_cong_veh_shed[zone]
        total_vkt += vkt_veh_shed[zone]

    moef.write("{0:.2f}%,".format(cong_vkt/total_vkt*100))
moef.write("\n")                             
moef.write( "Truck,")
moef.write( "{0:.2f}%\n".format(cong_truck_pkt/total_truck_pkt*100))

#"04	Congestion is reduced, enabling people to accomplish more because they spend less time on travel	"
moef.write("01	% of off peak road network congested 	\n")

total_auto_pkt = 0.0
total_truck_pkt = 0.0

cong_auto_pkt = 0.0
cong_truck_pkt = 0.0

for link in ntwk_md.links():
    #ignore centroid connectors, transfer links
    if link.length > 0 and link.data3!= 9999 and link.auto_volume >0:
        link_veh = 0.0
        link_truck = 0.0
        for volume in veh_attributes:
            link_veh += link[volume]
        for volume in truck_attributes:
            link_truck += link[volume]
        total_auto_pkt += link_veh*link.length
        total_truck_pkt += link_truck*link.length
        
        try:
            if (link_veh+link_truck)/(link.num_lanes*link.data3) > cong_frac:
                cong_auto_pkt += link_veh*link.length
                cong_truck_pkt += link_truck*link.length
        except:
            print link.id

moef.write( "Vehicle,")
moef.write( "{0:.2f}%\n".format(cong_auto_pkt/total_auto_pkt*100))
moef.write( "Truck,")
moef.write( "{0:.2f}%\n".format(cong_truck_pkt/total_truck_pkt*100))



In [55]:
moef.write(" average travel time for commuting trips,")

#exp_time = matrix_list[hbw_demand_pk] + "*(" + min_transit_time_peak + ".min." + matrix_list[sov_time_am] + ")"

exp_transit = "(" + matrix_list[hbw_local_bus_drive_demand_pk]\
    + "+" + matrix_list[hbw_local_bus_walk_demand_pk]\
    + "+" + matrix_list[hbw_rapid_bus_drive_demand_pk]\
    + "+" + matrix_list[hbw_rapid_bus_walk_demand_pk]\
    + "+" + matrix_list[hbw_express_bus_demand_pk]\
    + "+" + matrix_list[hbw_express_bus_walk_demand_pk]\
    + "+" + matrix_list[hbw_subway_knr_demand_pk]\
    + "+" + matrix_list[hbw_subway_pnr_demand_pk]\
    + "+" + matrix_list[hbw_subway_tr_demand_pk]\
    + "+" + matrix_list[hbw_subway_wk_demand_pk]\
    + "+" + matrix_list[hbw_train_knr_demand_pk]\
    + "+" + matrix_list[hbw_train_pnr_demand_pk]\
    + "+" + matrix_list[hbw_train_tr_demand_pk]\
    + "+" + matrix_list[hbw_train_wk_demand_pk] + ")"

temp_transit = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = temp_transit,
              matrix_name= "temp_transit",
              matrix_description = "temp_transit")

spec = {
    "type": "MATRIX_CALCULATION",
    "result": temp_transit,
    "expression":  exp_transit
}
matrix_calculator(spec)
    
exp_sov = "(" + matrix_list[hbw_auto1_nt_demand_pk]\
    + "+" + matrix_list[hbw_auto1_toll_demand_pk] + ")"

temp_sov = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = temp_sov,
              matrix_name= "temp_sov",
              matrix_description = "temp_sov")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": temp_sov,
    "expression":  exp_sov
}
matrix_calculator(spec)
    
exp_hov2 = "(" + matrix_list[hbw_auto2_nt_hov_demand_pk]\
    + "+" + matrix_list[hbw_auto2_nt_nh_demand_pk]\
    + "+" + matrix_list[hbw_auto2_toll_hov_demand_pk]\
    + "+" + matrix_list[hbw_auto2_toll_nh_demand_pk] + ")"

temp_hov2 = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = temp_hov2,
              matrix_name= "temp_hov2",
              matrix_description = "temp_hov2")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": temp_hov2,
    "expression":  exp_hov2
}
matrix_calculator(spec)
    
exp_hov3 = "(" + matrix_list[hbw_auto3_nt_hov_demand_pk]\
    + "+" + matrix_list[hbw_auto3_nt_nh_demand_pk]\
    + "+" + matrix_list[hbw_auto3_toll_hov_demand_pk]\
    + "+" + matrix_list[hbw_auto3_toll_nh_demand_pk] + ")"

temp_hov3 = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = temp_hov3,
              matrix_name= "temp_hov3",
              matrix_description = "temp_hov3")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": temp_hov3,
    "expression":  exp_hov3
}
matrix_calculator(spec)
    
exp_time = min_transit_time_peak + "*" + temp_transit\
    + "+" + temp_sov + "*" + matrix_list[sov_time_am]\
    + "+" + temp_hov2 + "*" + matrix_list[hov2_time_am]\
    + "+" + temp_hov3 + "*" + matrix_list[hov3_time_am]\
    + "+" + matrix_list[hbw_bike_demand_pk] + "*" + matrix_list[active_time]\
    + "+" + matrix_list[hbw_walk_demand_pk] + "*" + matrix_list[active_time] + "*5"
    
exp_total = "(" + temp_transit + "+" + temp_sov + "+" + temp_hov2 + "+" + temp_hov3 + ")"
   
MOE_fraction(moef,exp_time,exp_total,zones_array)

moef.write(" transit travel time for commuting trips,")

exp_time = "(" + exp_total + "*" + min_transit_time_peak + ")"

MOE_fraction(moef,exp_time,exp_total,zones_array)


moef.write( "% active mode share,")

MOE_fraction(moef,matrix_list[bike_demand_pk] + "+" + matrix_list[walk_demand_pk], exp_pk_pers_trips ,zones_array)

moef.write( "% transit mode share,")

MOE_fraction(moef,matrix_list[transit_demand_pk], exp_pk_pers_trips ,zones_array)

In [56]:
moef.close()

In [211]:
print "03	Major employment areas are supported by a full range of transportation options	" #fix
print "01	 Average travel time to nearest major employment areas from place of residence"

zones_array =["all"]

min_dist = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = min_dist,
    matrix_name= "min.dist",
    matrix_description = "min.dist")
closest_map = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = closest_map,
    matrix_name= "closest.map",
    matrix_description = "closest.map")

spec = common.get_mc_spec(min_dist, matrix_list[sov_dist_am],None,".min.","all",zones_employment,None)
matrix_calculator(spec)

spec = common.get_mc_spec(closest_map, min_dist + "==" + min_dist,None,None,"all",zones_employment,None)
matrix_calculator(spec)

expression_time = "(" + closest_map + "*(" + min_transit_time_peak + ".min." + matrix_list[sov_time_am] + "))*"\
    + pop_o_matrix
expression_total = pop_o_matrix + "*" + closest_map
for origins in zones_array:
    spec = common.get_mc_spec(None, expression_total,"+","+",origins,"all",None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_time,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    print origins + ": " + "{0:.2f}".format(top/bottom)
eb.delete_matrix(min_dist)
eb.delete_matrix(closest_map)


03	Major employment areas are supported by a full range of transportation options	
01	 Average travel time to nearest major employment areas from place of residence
all: 0.00


In [131]:
print "02	% of people and jobs accessible from goods movement centres (including themselves) 	"
           
expression_access =  matrix_list[hov3_time_am]  + " <45"



access_destinations = eb.available_matrix_identifier('DESTINATION')
create_matrix(matrix_id = access_destinations,
    matrix_name= "acessible",
    matrix_description = "acessible")

spec = common.get_mc_spec(access_destinations, expression_access,".max.",None,"4292;3078;5171","all",None)
matrix_calculator(spec)

expression_access = "((" + pop_d_matrix + "+" + emp_d_matrix + ")*" +access_destinations + ")"
print expression_access
expression_total = "(" + pop_d_matrix + "+" + emp_d_matrix + ")"
print expression_total


spec = common.get_mc_spec(None, expression_access,None,"+",None,"all",None)
access = matrix_calculator(spec)['result']
spec = common.get_mc_spec(None, expression_total,None,"+",None,"all",None)
total = matrix_calculator(spec)['result']
print "{0:.2f}%".format(access/total*100)

eb.delete_matrix(access_destinations)


02	% of people and jobs accessible from goods movement centres (including themselves) 	
((md1+md2)*md20)
(md1+md2)
71.08%


In [101]:
print "04	Congestion is reduced, enabling people to accomplish more because they spend less time on travel	"
print "01	% of peak road network congested 	"

cong_frac =0.85

total_auto_pkt = 0.0
total_truck_pkt = 0.0

cong_auto_pkt = 0.0
cong_truck_pkt = 0.0

veh_attributes = ["@volau1_nt","@volau1_toll",
                  "@volau2_nt_hov","@volau2_nt_nh","@volau2_toll_hov","@volau2_toll_nh",
                  "@volau3_nt_hov","@volau3_nt_nh","@volau3_toll_hov","@volau3_toll_nh"]

truck_attributes = ["@volau_htruck","@volau_ltruck","@volau_mtruck"]

for link in ntwk_md.links():
    #ignore centroid connectors, transfer links
    if link.length > 0 and link.data3!= 9999 and link.data3 >0:
        link_veh = 0.0
        link_truck = 0.0
        for volume in veh_attributes:
            link_veh += link[volume]
        for volume in truck_attributes:
            link_truck += link[volume]
        total_auto_pkt += link_veh*link.length
        total_truck_pkt += link_truck*link.length
        
        if (link_veh+link_truck)/(link.num_lanes*link.data3) > cong_frac:
            cong_auto_pkt += link_veh*link.length
            cong_truck_pkt += link_truck*link.length

print "Vehicle"
print "{0:.2f}%".format(cong_auto_pkt/total_auto_pkt*100)
print "Truck"
print "{0:.2f}%".format(cong_truck_pkt/total_truck_pkt*100)

04	Congestion is reduced, enabling people to accomplish more because they spend less time on travel	
01	% of peak road network congested 	
Vehicle
2.02%
Truck
1.05%


In [ ]:
print "01	Connected	"
print "01	People and goods are able to move efficiently with minimal delay 	"
print "01	Average travel time for people"

zones_array =["all", zones_inner, zones_outer,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(exp_pk_pers_trip_time,exp_pk_pers_trips,zones_array)

print "02	Average travel time for goods"

print "GGH Wide"

spec = common.get_mc_spec(None, exp_pk_trk_trips,"+","+",None,None,None)
total_truck_trips = matrix_calculator(spec)['result']
expression_string = "(" + matrix_list[truck_l_demand_am] + "+" + matrix_list[truck_m_demand_am]\
    + "+" + matrix_list[truck_h_demand_am]+ ")*" + matrix_list[sov_time_am]
spec = common.get_mc_spec(None, expression_string,"+","+",None,None,None)
total_truck_trip_time = matrix_calculator(spec)['result']
print total_truck_trip_time/total_truck_trips

print "02	It is easy to travel across the region by transit, and connections between the transit systems are seamless"
print "01	Average transit travel time per trip"

expression_total = matrix_list[transit_2hr_demand_am]
expression_time = min_transit_time_peak + "*" + matrix_list[transit_2hr_demand_am]

zones_array =[zones_ugc, zones_inner, zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_time,expression_total,zones_array)

print "02	Transit mode share for interregional  passenger travel"
print "03	There is a high level of transit service that is easy for people to get to"
print "01	% of people and jobs in the GGH accessible by transit within 45 min	"

expression_access = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + min_transit_time_offpeak  + " <45)"
expression_total = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")"

zones_array =["all", zones_inner, zones_outer,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_access,expression_total,zones_array)

print "04	A well connected road and highway network supports the movement of people and goods throughout the region"
print "01	% of people and jobs in the GGH accessible on the road network (no transit) within 45 min"

expression_access = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + matrix_list[hov3_time_am]  + " <45)"
expression_total = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")"

zones_array =["all", zones_inner, zones_outer,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_access,expression_total,zones_array)

print "02	% of people and jobs accessible from goods movement centres (including themselves) 	"
           
expression_access = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + matrix_list[hov3_time_am]  + " <45)"
expression_total = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")"

print  "Aggregate employment lands (weighted average based on number of jobs in that employment"
spec = common.get_mc_spec(None, expression_access,"+","+",zones_employment,"all",None)
access = matrix_calculator(spec)['result']
spec = common.get_mc_spec(None, expression_total,"+","+",zones_employment,"all",None)
total = matrix_calculator(spec)['result']
print "{0:.2f}%".format(access/total*100)

print "05	A regional network that supports high occupancy vehicle use (e.g. carpooling, transit)	"
print "01	Average HOV travel time"

expression_time = matrix_list[hov2_demand_am] + "*" + matrix_list[hov2_time_am] + "+" + matrix_list[hov3_demand_am] + "*" + matrix_list[hov3_time_am]
expression_total = matrix_list[hov2_demand_am] + "+" + matrix_list[hov3_demand_am]

zones_array =["all", zones_inner, zones_outer,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_total,expression_time,zones_array)

print "02	HOV mode share"
expression_hov = matrix_list[hov2_demand_pk]+"*"+str(hov2_oc)+"+"+ matrix_list[hov3_demand_pk]+ "*"+str(hov3_oc)

zones_array =["all", zones_inner, zones_outer,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_hov,exp_pk_pers_trips,zones_array)

print "06	It is easy to travel between urban and rural areas by a range of transportation options "
print "(road, transit, cycling, water, rail, etc.)	"
print "01	% of people and jobs accessible within 45 min from the rural areas	"

expression_access = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + exp_pk_min_travel_time  + " <45)"
expression_total = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")"

zones_array =[zones_rural,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_access,expression_total,zones_array)

print "02	Integrated	"
print "01	People live in complete communities where access to services, jobs and recreation is close and convenient"
print "01	Auto Person kilometres travelled (PKT) per capita"

expression  = "(" + matrix_list[hov2_demand_pk] + "*"+ str(hov2_oc)\
    + "+" + matrix_list[hov3_demand_pk] + "*" + str(hov3_oc)\
    + "+" + matrix_list[sov_demand_pk] + "*" + str(sov_oc)\
    + "+" + matrix_list[bike_demand_pk] + "+" + matrix_list[transit_demand_pk] \
    + "+" + matrix_list[walk_demand_pk] + ")*" + matrix_list[sov_dist_am]

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]
    
for origins in zones_array:
    spec = common.get_mc_spec(None, pop_o_matrix,"+",None,origins,None,None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    print origins + ": " + "{0:.2f}".format(top/bottom)

print "02	Average travel distance for all trips peak"

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression,exp_pk_pers_trips,zones_array)

print "03	Average travel distance for all trips off peak 	"

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression,exp_offpk_pers_trips,zones_array)


print "03	People have easy access to  jobs and multiple transportation options to get there. "
print "01	% of jobs accessible in less than 45 min  by road"

expression_access = pop_o_matrix + "*" + emp_d_matrix + "*(" + matrix_list[sov_time_am]  + " <45)"
expression_total = pop_o_matrix + "*" + emp_d_matrix

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_access,expression_total,zones_array)

print "02	% of jobs accessible in less than 45 min  by transit 	"
expression_access = pop_o_matrix + "*" + emp_d_matrix + "*(" + min_transit_time_peak  + " <45)"
expression_total = pop_o_matrix + "*" + emp_d_matrix

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_access,expression_total,zones_array)

print "03	% of jobs accessible in less than 45 min  by active modes	"
expression_access = pop_o_matrix + "*" + emp_d_matrix + "*(" + matrix_list[active_time]  + " <45)"
expression_total = pop_o_matrix + "*" + emp_d_matrix

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_access,expression_total,zones_array)

print "04	Major good movement related industries are supported by an efficient multi-modal system of "
print "roads, rail, ports and airports	"
print "01	% of truck trips originating or ending"
print "intermodal hubs, ports, airports and border crossings less than 45 minutes long	"
print "For logistic centres"

print "03	Equitable	"
print "01	Reliable and frequent transit services can be accessed by all users regardless of age, income, or ability."
print "01	Percent of people and jobs accessible by transit for low income"

expression_access = low_inc_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + min_transit_time_peak  + " <45)"
expression_total = low_inc_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")"

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_access,expression_total,zones_array)

print "02	Percent of people and jobs accessible by transit for youth 	"
expression_access = youth_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + min_transit_time_peak  + " <45)"
expression_total = youth_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")"

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_access,expression_total,zones_array)

print "03	Percent of people accessible by transit for seniors	"
expression_access = senior_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + min_transit_time_peak  + " <45)"
expression_total = senior_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")"

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_access,expression_total,zones_array)

print "04	Transportation costs per capita	"
print "Inner"
print "Outer"
print "Area Sheds"

print "02	Reliable and frequent transit services are available to all users regardless of where they live"
print "01	% of people and jobs in the GGH accessible by transit within 45 min"

expression_access = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")*(" + min_transit_time_peak  + " <45)"
expression_total = pop_o_matrix + "*(" + pop_d_matrix + "+" + emp_d_matrix + ")"

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

MOE_fraction(expression_access,expression_total,zones_array)

print "03	The transportation system provides efficient access to services such as colleges and hospitals	"
print "01	% of residents that can access a major hospital in less than 45 by non auto mode"

expression_access = "(" + min_transit_time_peak + ".min." + matrix_list[active_time] + ")<45"

zones_array =[zones_inner,zones_outer,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

access_origins = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = access_origins,
    matrix_name= "acessible",
    matrix_description = "acessible")

spec = common.get_mc_spec(access_origins, expression_access,None,".max.","all",zones_hospitals,None)
matrix_calculator(spec)
expression_access = "(" + pop_o_matrix + "*" + access_origins + ")"
expression_total = pop_o_matrix 
for origins in zones_array:
    spec = common.get_mc_spec(None, expression_total,"+",None,origins,None,None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_access,"+",None,origins,None,None)
    top = matrix_calculator(spec)['result']
    print origins + ": " + "{0:.2f}".format(top/bottom)
eb.delete_matrix(access_origins)

print "02 % of residents that can access a post secondary institution in less than 45 min by non auto mode"
expression_access = "(" + min_transit_time_peak + ".min." + matrix_list[active_time] + ")<45"

zones_array =[zones_inner,zones_outer,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

access_origins = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = access_origins,
    matrix_name= "acessible",
    matrix_description = "acessible")

spec = common.get_mc_spec(access_origins, expression_access,None,".max.","all",zones_university,None)
matrix_calculator(spec)
expression_access = "(" + pop_o_matrix + "*" +access_origins + ")"
expression_total = pop_o_matrix 
for origins in zones_array:
    spec = common.get_mc_spec(None, expression_total,"+",None,origins,None,None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_access,"+",None,origins,None,None)
    top = matrix_calculator(spec)['result']
    print origins + ": " + "{0:.2f}".format(top/bottom)
eb.delete_matrix(access_origins)

print "04	Environmentally Sustainable	"
print "01	The transportation system supports reduced energy use and reductions in greenhouse gas emissions"
print "01	Total tonnes of GHG emitted per year per capita" 
            #GGH Wide
        #02	GHG intensity per VKT (g CO2e per km)
            #GGH Wide
    #02	The preservation of valuable resources, natural areas and wildlife is prioritized in planning 
    #and use of the transportation system 	
        #01	Route km of new transportation corridor of environmentally sensitive areas 	
            #GGH Wide
        #02	Route-km of new transportation corridors through protected countryside (Green Belt)	
            #GGH Wide
        #03	# of new or expanded transportation corridors crossing major watercourse 	
            #GGH Wide
    #03	Impacts on prime agricultural lands are minimized in planning and use of the transportation system	
        #04	Route-km of new transportation corridors crossing agricultural lands	
            #GGH Wide
print "05	Economically Sustainable	"
    #01	Transportation investments make efficient use of financial resources	
        #01	Annualized costs of transportation infrastructure per PKT 	
            #GGH Wide
    #03	The transportation system is affordable to operate and maintain	0
        #01	Operating and maintenance costs per trip
            #GGH Wide
print "02	PKT demand/PKT capacity for all motorized modes" 
print "(for private modes, capacity will be derived from vehicle lane capacity multiplied by auto occupancy)"
print "GGH Wide"
total_cap_lane_km = 0.0
total_person_lane_km = 0.0

for link in ntwk_am.links():
    #ignore centroid connectors
    if link.volume_delay_func != 90:
        #auto persons
        link_persons = (link["@volau1_nt"] + link["@volau1_toll"])*sov_oc \
        + ( link["@volau2_nt_hov"] + link["@volau2_nt_nh"] + link["@volau2_toll_hov"] + link["@volau2_toll_nh"])*hov2_oc \
        + ( link["@volau3_nt_hov"] + link["@volau3_nt_nh"] + link["@volau3_toll_hov"] + link["@volau3_toll_nh"])*hov3_oc
        
        #check if hov
        if link['@is_hov2'] == 0 and link['@is_hov3'] == 0:
            link_cap = link.data3*sov_oc
        else:
            link_cap = link.data3*(link['@is_hov2']*hov2_oc + link['@is_hov3']*hov3_oc)
        
        #transit
        for segment in link.segments():
            transit_line = segment.line
            link_cap += 60 * num_hrs_am / transit_line.headway * transit_line.vehicle.total_capacity
            link_persons += segment.transit_volume                
        if link_persons >0:
            total_cap_lane_km += link_cap*link.length*link.num_lanes
            total_person_lane_km += link_persons*link.length*link.num_lanes
print total_person_lane_km/total_cap_lane_km

print "06	Active, Safe and Healthy	"
print "01	The transportation system supports walking and cycling, reducing dependence on personal vehicles	"
print "01	% of all trips involving active transportation (walking and cycling)	"

expression_active = matrix_list[walk_demand_pk] + "+" + matrix_list[bike_demand_pk]

zones_array =[zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]


MOE_fraction(expression_active,exp_pk_pers_trips,zones_array)

print "02	VKT per capita	"

zones_array = ["all",shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]
expression_total_vkt = "(" + matrix_list[sov_demand_pk] + "+" + matrix_list[hov2_demand_pk]\
                        + "+" + matrix_list[hov3_demand_pk] + ")*" + matrix_list[sov_dist_am]
    
for origins in zones_array:
    spec = common.get_mc_spec(None, pop_o_matrix,"+",None,origins,None,None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_total_vkt,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    print origins + ": " + "{0:.2f}".format(top/bottom)
    
    #02	The transportation system’s  impact on air quality is reduced 	
        #01	Total NOx and VOC emissions (tonnes per year)
            #GGH Wide
        #02	Total particulate matter emissions (tonnes PM2,5  per year)
            #GGH Wide
        #03	Exposure to NOx and VOC per capita within 500 metres of an expressway and highway 
        #(g (NOx+VOC) per capita per year)
            #GGH Wide
        #04	Exposure to PM2.5 per capita within 500 metres of an expressway and highway 
        #(g PM2.5 per capita per year)
            #GGH Wide
    #04	The transportation system minimizes accidents and injuries	
        #01	% change in number of collisions compared to BAU  (collisions identified from VKT by mode)
print "07	Resilient	"
print "01	The transportation system is designed and constructed to manage the risks associated with climate change"
print "(e.g., changing engineering standards so infrastructure can withstand more extreme weather events)"
        #01	"Route-km on roads through areas susceptible to blowing snow
            #GGH Wide
print "08	Prosperous	"
print "01	The transportation system supports business and trade by ensuring efficient travel times to, from "
print "and through gateways to the region (ports, airports, border crossings)	"
print "01	Average truck travel times to and from intermodal hubs/ports/airports/ and gateways"


print "02	The transportation system promotes global travel, goods movement  and competitiveness "
print "by linking people and businesses  to airports in the region 	"
print "01	% of people and jobs within 45 minute travel time to major international airports for person trips 	"

expression_access = "(" + min_transit_time_peak + ".min." + matrix_list[sov_time_am] + ")<45"

zones_array =["all",\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

access_origins = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = access_origins,
    matrix_name= "acessible",
    matrix_description = "acessible")

spec = common.get_mc_spec(access_origins, expression_access,None,".max.","all",zones_airports,None)
matrix_calculator(spec)
expression_access = "(" + pop_o_matrix + "*" +access_origins + ")"
expression_total = pop_o_matrix 
for origins in zones_array:
    spec = common.get_mc_spec(None, expression_total,"+",None,origins,None,None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_access,"+",None,origins,None,None)
    top = matrix_calculator(spec)['result']
    print origins + ": " + "{0:.2f}".format(top/bottom)
eb.delete_matrix(access_origins)

print "02	% of people and jobs within 45 minute travel time to airports for truck trips"
expression_access = "(" + matrix_list[sov_time_am] + ")<45"

zones_array =["all",\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

access_origins = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = access_origins,
    matrix_name= "acessible",
    matrix_description = "acessible")

spec = common.get_mc_spec(access_origins, expression_access,None,".max.","all",zones_airports,None)
matrix_calculator(spec)
expression_access = "(" + pop_o_matrix + "+" + emp_o_matrix + ")*" +access_origins
expression_total = pop_o_matrix + "+" + emp_o_matrix
for origins in zones_array:
    spec = common.get_mc_spec(None, expression_total,"+",None,origins,None,None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_access,"+",None,origins,None,None)
    top = matrix_calculator(spec)['result']
    print origins + ": " + "{0:.2f}".format(top/bottom)
eb.delete_matrix(access_origins)

print "03	Major employment areas are supported by a full range of transportation options	"
print "01	 Average travel time to nearest major employment areas from place of residence"

zones_array =["all",zones_ugc,zones_inner,zones_outer,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

min_dist = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = min_dist,
    matrix_name= "min.dist",
    matrix_description = "min.dist")
closest_map = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = closest_map,
    matrix_name= "closest.map",
    matrix_description = "closest.map")

spec = common.get_mc_spec(min_dist, matrix_list[sov_dist_am],None,".min.","all",zones_airports,None)
matrix_calculator(spec)

spec = common.get_mc_spec(closest_map, min_dist + "==" + min_dist,None,None,"all",zones_airports,None)
matrix_calculator(spec)

expression_time = "(" + closest_map + "*(" + min_transit_time_peak + ".min." + matrix_list[sov_time_am] + "))*"\
    + pop_o_matrix
expression_total = pop_o_matrix + "*" + closest_map
for origins in zones_array:
    spec = common.get_mc_spec(None, expression_total,"+","+",origins,"all",None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_time,"+","+",origins,"all",None)
    top = matrix_calculator(spec)['result']
    print origins + ": " + "{0:.2f}".format(top/bottom)
eb.delete_matrix(min_dist)
eb.delete_matrix(closest_map)
            
print "02	% of residents that can access  major employment areas in less than 45 minutes	"

expression_access = "(" + matrix_list[sov_time_am] + ".min." + min_transit_time_peak + ")<45"

zones_array =["all",zones_ugc,zones_inner,zones_outer,zones_pd1,\
              shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north]

access_origins = eb.available_matrix_identifier('ORIGIN')
create_matrix(matrix_id = access_origins,
    matrix_name= "acessible",
    matrix_description = "acessible")

spec = common.get_mc_spec(access_origins, expression_access,None,".max.","all",zones_airports,None)
matrix_calculator(spec)
expression_access = "(" + pop_o_matrix + "*" + access_origins + ")"
expression_total = pop_o_matrix 
for origins in zones_array:
    spec = common.get_mc_spec(None, expression_total,"+",None,origins,None,None)
    bottom = matrix_calculator(spec)['result']
    spec = common.get_mc_spec(None, expression_access,"+",None,origins,None,None)
    top = matrix_calculator(spec)['result']
    print origins + ": " + "{0:.2f}".format(top/bottom)
eb.delete_matrix(access_origins)

print "04	Congestion is reduced, enabling people to accomplish more because they spend less time on travel	"
print "01	% of road network congested 	"
print "Distinguish by direction	"

cong_frac =0.85

total_lane_km = 0.0
cong_lane_km = 0.0

central_total_lane_km = 0.0
central_cong_lane_km = 0.0
east_total_lane_km = 0.0
east_cong_lane_km = 0.0
southwest_total_lane_km = 0.0
southwest_cong_lane_km = 0.0
midwest_total_lane_km = 0.0
midwest_cong_lane_km = 0.0
west_total_lane_km = 0.0
west_cong_lane_km = 0.0
north_total_lane_km = 0.0
north_cong_lane_km = 0.0

vol_attributes = ["@volau1_nt","@volau1_toll",
                  "@volau2_nt_hov","@volau2_nt_nh","@volau2_toll_hov","@volau2_toll_nh",
                  "@volau3_nt_hov","@volau3_nt_nh","@volau3_toll_hov","@volau3_toll_nh",
                  "@volau_htruck","@volau_ltruck","@volau_mtruck"]

types_central =[101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,329,331,333]
types_east = [217,218,219,220,221,222,223,224,900,910,930]
types_midwest = [434,435,436,537,538,539,540]
types_southwest = [641,642,643,644,645,646,700,800,810]
types_west = [820,850]
types_north = [325,326,327,328,330,332,870,880]

for link in ntwk_am.links():
    #ignore centroid connectors, transfer links
    if link.length > 0 and link.data3!= 9999 and link.data3 >0:
        link_veh = 0.0
        for volume in vol_attributes:
            link_veh += link[volume]
        total_lane_km += link.num_lanes*link.length
        if link.type in types_central:
            central_total_lane_km += link.num_lanes*link.length
        elif link.type in types_east:
            east_total_lane_km += link.num_lanes*link.length
        elif link.type in types_southwest:
            southwest_total_lane_km += link.num_lanes*link.length
        elif link.type in types_midwest:
            midwest_total_lane_km += link.num_lanes*link.length
        elif link.type in types_west:
            west_total_lane_km += link.num_lanes*link.length
        elif link.type in types_north:
            north_total_lane_km += link.num_lanes*link.length
        if link_veh/(link.num_lanes*link.data3) > cong_frac:
            cong_lane_km += link.num_lanes*link.length
            if link.type in types_central:
                central_cong_lane_km += link.num_lanes*link.length
            elif link.type in types_east:
                east_cong_lane_km += link.num_lanes*link.length
            elif link.type in types_southwest:
                southwest_cong_lane_km += link.num_lanes*link.length
            elif link.type in types_midwest:
                midwest_cong_lane_km += link.num_lanes*link.length
            elif link.type in types_west:
                west_cong_lane_km += link.num_lanes*link.length
            elif link.type in types_north:
                north_cong_lane_km += link.num_lanes*link.length

print "GGH Wide"
print "{0:.2f}%".format(cong_lane_km/total_lane_km*100)
print "Shed - Central"
print "{0:.2f}%".format(central_cong_lane_km/central_total_lane_km*100)
print "Shed - East"
print "{0:.2f}%".format(east_cong_lane_km/east_total_lane_km*100)
print "Shed - Southwest"
print "{0:.2f}%".format(southwest_cong_lane_km/southwest_total_lane_km*100)
print "Shed - Midwest"
print "{0:.2f}%".format(midwest_cong_lane_km/midwest_total_lane_km*100)
print "Shed - West"
print "{0:.2f}%".format(west_cong_lane_km/west_total_lane_km*100)
print "Shed - North"
print "{0:.2f}%".format(north_cong_lane_km/north_total_lane_km*100)

print "02	Travel time savings per trip for people (roads and transit)"

zones_array =["all"]
MOE_fraction(exp_pk_pers_trip_time,exp_pk_pers_trips,zones_array)

print "03	Travel time savings per trip for goods"
zones_array =["all"]

expression_truck_time = "(" + exp_pk_trk_trips + ")*" + matrix_list[sov_time_am]
MOE_fraction(expression_truck_time,exp_pk_trk_trips,zones_array)            
print "04	Cost of congestion - all persons travelling on motorized vehicles"
            #GGH Wide


01	Connected	
01	People and goods are able to move efficiently with minimal delay 	
01	Average travel time for people
all: 14.41
gc1: 14.06
gc2: 15.51
gs1: 14.13
gs2: 17.82
gs3: 13.63
gs4: 13.54
gs5: 12.24
gs6: 17.01
02	Average travel time for goods
GGH Wide
23.7359773086
02	It is easy to travel across the region by transit, and connections between the transit systems are seamless
01	Average transit travel time per trip
gb4: 18.19
gc1: 33.66
gc2: 41.90
gf1: 12.46
gs1: 27.14
gs2: 62.03
gs3: 54.51
gs4: 41.70
gs5: 33.06
gs6: 53.35
02	Transit mode share for interregional  passenger travel
03	There is a high level of transit service that is easy for people to get to
01	% of people and jobs in the GGH accessible by transit within 45 min	
all: 0.17
gc1: 0.21
gc2: 0.05
gs1: 0.28
gs2: 0.07
gs3: 0.19
gs4: 0.05
gs5: 0.06
gs6: 0.08
04	A well connected road and highway network supports the movement of people and goods throughout the region
01	% of people and jobs in the GGH accessible on the road n

In [38]:
save_path = "C:\\Users\\mnasterska\\Documents\\Projects\\GGH\\MOE"
with open (os.path.join(save_path, scenario_folder.replace("\\","_").replace(" ", "_") +".csv"),"wb") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter = ",", quoting = csv.QUOTE_MINIMAL)
    csvwriter.writerow(["MOE Number", "Value"])
    for line in MOEs:
        csvwriter.writerow([line, MOEs[line]])

In [31]:
moe.MOE_RWDI(ntwk_am, eb)
moe.MOE_RWDI(ntwk_md, eb)

Total Freeway VKT is: 11,950,399
Total Highway VKT is: 11,572,416
Total Arterial VKT is: 7,175,550
Total Collector VKT is: 3,012,456
Average Freeway Speed is: 67.36
Average Highway Speed is: 59.04
Average Arterial Speed is: 37.99
Average Collector Speed is: 36.85
12.11% of freeway vehicles are light/medium trucks
9.85% of freeway vehicles are heavy trucks
0.05% of freeway vehicles are buses
10.97% of highway vehicles are light/medium trucks
4.02% of highway vehicles are heavy trucks
0.07% of highway vehicles are buses
9.26% of arterial vehicles are light/medium trucks
2.71% of arterial vehicles are heavy trucks
0.49% of arterial vehicles are buses
8.14% of collector vehicles are light/medium trucks
2.25% of collector vehicles are heavy trucks
0.46% of collector vehicles are buses
Total Freeway VKT is: 7,067,998
Total Highway VKT is: 5,306,421
Total Arterial VKT is: 3,067,940
Total Collector VKT is: 1,207,625
Average Freeway Speed is: 92.29
Average Highway Speed is: 70.90
Average Arteri